In [168]:
import os
import re
import aocd
import string
import numpy as np
import pandas as pd
from copy import deepcopy
from aocd import submit

digits = ['0','1','2','3','4','5','6','7','8','9']

data = aocd.get_data(day=5, year=2023).split('\n')
data

['seeds: 3037945983 743948277 2623786093 391282324 195281306 62641412 769611781 377903357 2392990228 144218002 1179463071 45174621 2129467491 226193957 1994898626 92402726 1555863421 340215202 426882817 207194644',
 '',
 'seed-to-soil map:',
 '3078006360 2182201339 30483272',
 '803630304 624445326 165226844',
 '2393736333 2745251526 281120946',
 '717936870 789672170 85693434',
 '598717319 410599330 27984688',
 '3999095007 2024628810 157572529',
 '3605588191 3026372472 22322803',
 '3555659576 2678166775 3396919',
 '968857148 438584018 1780307',
 '3216227818 2212684611 87459567',
 '2302084376 4122083708 91651957',
 '970637455 0 188112122',
 '507182228 299146916 40412346',
 '1372302034 1689624457 202945009',
 '1370123632 191483770 2178402',
 '324787204 193662172 105484744',
 '3116425470 2671328191 6838584',
 '626702007 875365604 82756204',
 '1575247043 978774853 317322423',
 '3134996187 4213735665 81231631',
 '2024628810 2681563694 63687832',
 '714565222 188112122 3371648',
 '547594574 16

In [93]:
data = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".split('\n')

In [169]:
# Part 1 
seed_nums = data[0].split(':')[-1].strip().split(' ')
seed_nums = [int(x) for x in seed_nums]
maps = []
_map = []
map_no = -1
for line in data[2:]:
    
    
    if not len(line):
        continue
        
    if '-to-' in line:
        _map = sorted(_map, key = lambda x: x[1])
        maps += [_map]
        map_name = '-to-'
        map_no += 1
        _map = []
    else:
        _map += [tuple(int(x) for x in line.split(' '))]
        
_map = sorted(_map, key = lambda x: x[1])
maps += [_map]

maps = maps[1:]
maps

[[(970637455, 0, 188112122),
  (714565222, 188112122, 3371648),
  (1370123632, 191483770, 2178402),
  (324787204, 193662172, 105484744),
  (507182228, 299146916, 40412346),
  (1158749577, 339559262, 71040068),
  (598717319, 410599330, 27984688),
  (968857148, 438584018, 1780307),
  (1245096080, 440364325, 107170721),
  (430271948, 547535046, 76910280),
  (803630304, 624445326, 165226844),
  (717936870, 789672170, 85693434),
  (626702007, 875365604, 82756204),
  (1229789645, 958121808, 15306435),
  (709458211, 973428243, 5107011),
  (1352266801, 978535254, 239599),
  (1575247043, 978774853, 317322423),
  (0, 1296097276, 324787204),
  (547594574, 1620884480, 51122745),
  (1352506400, 1672007225, 17617232),
  (1372302034, 1689624457, 202945009),
  (3999095007, 2024628810, 157572529),
  (3078006360, 2182201339, 30483272),
  (3216227818, 2212684611, 87459567),
  (3627910994, 2300144178, 371184013),
  (3116425470, 2671328191, 6838584),
  (3555659576, 2678166775, 3396919),
  (2024628810, 2681

In [75]:
# Part 1
locations = []
for out in seed_nums:
    #print('Seed',out)
    map_no = 0
    for _map in maps:
        #print('Map', map_no)
        a = [x for x in _map if x[1] <= out and x[1] + x[2] > out]
        #print('  Found map',a)
        if not len(a):
            out = out
        else:
            out = a[0][0] + out - a[0][1]
        #print('  out', out)
        map_no += 1
    locations += [out]
print(locations)
print(sorted(locations)[0])
    

[2555693727, 2716845820, 3421775940, 1326688836, 3482819330, 4034180178, 3949698070, 1313309869, 1526086962, 4115756768, 3553393809, 4016713387, 3080164134, 226172555, 2841393735, 4063941492, 2672001563, 1277267466, 622612797, 3494732668]
226172555


In [164]:
# Part 2
seed_nums = data[0].split(':')[-1].strip().split(' ')
seed_nums = [int(x) for x in seed_nums]
seed_nums = np.array(seed_nums).reshape(-1,2)
seed_nums

array([[3037945983,  743948277],
       [2623786093,  391282324],
       [ 195281306,   62641412],
       [ 769611781,  377903357],
       [2392990228,  144218002],
       [1179463071,   45174621],
       [2129467491,  226193957],
       [1994898626,   92402726],
       [1555863421,  340215202],
       [ 426882817,  207194644]])

In [165]:
def get_output_ranges_given_input(start, _range, _map, _print=False):
    
    out_ranges = []
    for map_dest, map_source, map_range in _map:
        if _print:
            print('  out ranges',out_ranges)
            print('Map',map_dest, map_source, map_range)
        if map_source + map_range < start:
            if _print:
                print('Map not relevant')
            continue
            
        if map_source >= (start + _range):
            if _print:
                print('Whole range before map...')
            out_ranges += [[start, _range]]
            return out_ranges
        
        if map_source == start:
            if _print:
                print('Map starts at start')
                print('  start',start)
                print('  range',_range)
                print('  map',map_dest, map_source, map_range)
            if map_range > _range:
                out_ranges += [[map_dest, _range]]
                return out_ranges
            else:
                out_ranges += [[map_dest, map_range]]
                _range -= map_range
                start += map_range
            continue
        
        if map_source > start:
            if _print:
                print('Map starts after start')
                print('  start',start)
                print('  range',_range)
                print('  map',map_dest, map_source, map_range)
            # Set beginning of range to same values
            out_ranges += [[start, map_source - start]]
            _range = _range - (map_source - start)
            start = map_source
            if _print:
                print('  New start', start)
                print('  New Range', _range)
            
            dest_start = start - map_source + map_dest
            if _range <= map_range:
                out_ranges += [[dest_start, _range]]
                return out_ranges
            else:
                out_ranges += [[dest_start, map_range]]
                start = start + map_range
                _range = _range - map_range
            
            if _print:
                print('  Out ranges', out_ranges)
            continue
                
        if start > map_source:
            if _print:
                print('Start starts after map_start')
                print('  start',start)
                print('  range',_range)
            dest_start = start - map_source + map_dest
            map_range_to_end = map_range - (start - map_source)
            if _range <= map_range_to_end:
                out_ranges += [[dest_start, _range]]
                return out_ranges
            else:
                out_ranges += [[dest_start, map_range_to_end]]
                start = start + map_range_to_end
                _range = _range - map_range_to_end
            continue
                
    if _range > 0:
        out_ranges += [[start,_range]]
    
    return out_ranges
out = get_output_ranges_given_input(91,149,maps[0])
sum([x[1] for x in out]),out, maps[0]

(149,
 [[970637546, 149]],
 [(970637455, 0, 188112122),
  (714565222, 188112122, 3371648),
  (1370123632, 191483770, 2178402),
  (324787204, 193662172, 105484744),
  (507182228, 299146916, 40412346),
  (1158749577, 339559262, 71040068),
  (598717319, 410599330, 27984688),
  (968857148, 438584018, 1780307),
  (1245096080, 440364325, 107170721),
  (430271948, 547535046, 76910280),
  (803630304, 624445326, 165226844),
  (717936870, 789672170, 85693434),
  (626702007, 875365604, 82756204),
  (1229789645, 958121808, 15306435),
  (709458211, 973428243, 5107011),
  (1352266801, 978535254, 239599),
  (1575247043, 978774853, 317322423),
  (0, 1296097276, 324787204),
  (547594574, 1620884480, 51122745),
  (1352506400, 1672007225, 17617232),
  (1372302034, 1689624457, 202945009),
  (3999095007, 2024628810, 157572529),
  (3078006360, 2182201339, 30483272),
  (3216227818, 2212684611, 87459567),
  (3627910994, 2300144178, 371184013),
  (3116425470, 2671328191, 6838584),
  (3555659576, 2678166775, 33

In [166]:
locations = []
in_ranges = seed_nums

for _map in maps:
    out_ranges = []
    print('Map',_map)
    print('  In ranges',list(in_ranges))
    
    for start, _range in in_ranges:
        out_ranges += get_output_ranges_given_input(start,_range,_map)
    in_ranges = out_ranges
    print('  Out ranges', out_ranges)

seed_nums, out_ranges

Map [(970637455, 0, 188112122), (714565222, 188112122, 3371648), (1370123632, 191483770, 2178402), (324787204, 193662172, 105484744), (507182228, 299146916, 40412346), (1158749577, 339559262, 71040068), (598717319, 410599330, 27984688), (968857148, 438584018, 1780307), (1245096080, 440364325, 107170721), (430271948, 547535046, 76910280), (803630304, 624445326, 165226844), (717936870, 789672170, 85693434), (626702007, 875365604, 82756204), (1229789645, 958121808, 15306435), (709458211, 973428243, 5107011), (1352266801, 978535254, 239599), (1575247043, 978774853, 317322423), (0, 1296097276, 324787204), (547594574, 1620884480, 51122745), (1352506400, 1672007225, 17617232), (1372302034, 1689624457, 202945009), (3999095007, 2024628810, 157572529), (3078006360, 2182201339, 30483272), (3216227818, 2212684611, 87459567), (3627910994, 2300144178, 371184013), (3116425470, 2671328191, 6838584), (3555659576, 2678166775, 3396919), (2024628810, 2681563694, 63687832), (2393736333, 2745251526, 2811209

(array([[3037945983,  743948277],
        [2623786093,  391282324],
        [ 195281306,   62641412],
        [ 769611781,  377903357],
        [2392990228,  144218002],
        [1179463071,   45174621],
        [2129467491,  226193957],
        [1994898626,   92402726],
        [1555863421,  340215202],
        [ 426882817,  207194644]]),
 [[2555693727, 10749292],
  [2087101348, 31968589],
  [841166972, 24837438],
  [866004410, 3338612],
  [122674836, 11140716],
  [3010362479, 25869285],
  [3509035984, 16232307],
  [1885168928, 15735290],
  [538950114, 72116346],
  [283612297, 12529151],
  [935374526, 46531696],
  [3219656827, 11732133],
  [3204882405, 7935838],
  [830900502, 5124513],
  [1399208768, 12619828],
  [3872898071, 8256780],
  [2591216728, 19940366],
  [905706118, 26271489],
  [1621759801, 2385011],
  [3036231764, 3010161],
  [437599889, 21366897],
  [1509338677, 14318416],
  [1859942766, 21407978],
  [2446596231, 16803323],
  [3755741982, 6017842],
  [2630748748, 3070511],

In [167]:
sorted([x[0] for x in out_ranges])

[47909639,
 87324760,
 93839242,
 122674836,
 141917941,
 145492041,
 147672017,
 188169313,
 216635734,
 229878818,
 283612297,
 299076626,
 355434063,
 381221042,
 388851709,
 437599889,
 538950114,
 611066460,
 622612797,
 633594314,
 686362835,
 687082519,
 704786709,
 732514228,
 764851360,
 767545585,
 767785792,
 770846183,
 784845239,
 800228980,
 829601748,
 830900502,
 841166972,
 866004410,
 905706118,
 931977607,
 935374526,
 991558233,
 1012775294,
 1023073377,
 1042264614,
 1047552057,
 1095268310,
 1112977542,
 1121006696,
 1196348942,
 1252999423,
 1299784866,
 1300181659,
 1346005842,
 1361312277,
 1392314987,
 1399208768,
 1411828596,
 1427899978,
 1450737623,
 1509338677,
 1526086962,
 1547888548,
 1562212796,
 1600731034,
 1621759801,
 1624144812,
 1664787666,
 1687875045,
 1793246847,
 1819286809,
 1859942766,
 1881350744,
 1882023558,
 1885168928,
 1910771697,
 1920362408,
 2006851158,
 2054646885,
 2087101348,
 2119069937,
 2121699584,
 2125541364,
 2205955797,
 

In [156]:
_map = [(39, 0, 15), (0, 15, 37), (37, 52, 2)]
start = 81
_range = 14
get_output_ranges_given_input(start,_range,_map,_print=True)

  out ranges []
Map 39 0 15
Start starts after map_start
  start 81
  range 14
  out ranges [[120, -66]]
Map 0 15 37
Map starts at start
  start 15
  range 80
  map 0 15 37
  out ranges [[120, -66], [0, 37]]
Map 37 52 2
Map starts at start
  start 52
  range 43
  map 37 52 2


[[120, -66], [0, 37], [37, 2], [54, 41]]

In [171]:
from itertools import count
for k in count():
    print(k)
    break

0
